In [1]:
# imports
import os
import cv2
from sklearn.ensemble import BaggingClassifier
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense, Dropout, MaxPool2D
from keras.models import Sequential
#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
#from time import time
import pandas as pd
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
import math
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
#### variabili globali
#### GIUSEPPE
#image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
image_path = '..\\..\\Untitled_Folder'
#### FABIO
# image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
csv_path = 'HAM10000_metadata.csv'
csv_completo = 'dataframe_completo.csv'
csv_completo_2 = 'dataframe_completo_2.csv'



In [3]:
# dal dataframe ne estrae le immagini in un array, serve nel batch
def load_images(array):
    images = list()
    #print(array.iloc[0])
    for index in range(0,len(array)):
        #print(index.type())
        img_id = array.iloc[index]
        elem = os.path.join(image_path, img_id)
        elem = elem + ".jpg"
        img = cv2.imread(elem)
        # TODO va fatto un reshape, non capisco perché
        images.append(np.asarray(img)) #########################################################################################
    return np.asarray(images)

def my_train_batch(dataset, batch, epochs, classifier):
    size = len(dataset)
    epoch=0
    cl1 = k.models.load_model(classifier)
    # train del classificatore un batch alla volta
    while size >= batch:
        # To get n random rows 
        samples = dataset.sample(n = batch)
        # splitting test and validation
        X_train, X_val, y_train, y_val = train_test_split(samples["image_id"], samples["dx"], test_size=0.30)
        
        train_img = np.asarray(load_images(X_train))
        val_img = np.asarray(load_images(X_val))
        
        #1hot encoder
        enc = OneHotEncoder(sparse=False)
        y_train_shaped = np.array(y_train).reshape(-1,1)
        y_val_shaped = np.array(y_val).reshape(-1,1)
        enc.fit(y_train_shaped)
        y_train_one = enc.transform(y_train_shaped)
        y_val_one = enc.transform(y_val_shaped)
        
        cl1.fit(train_img, y_train_one, epochs=epochs, shuffle=True, validation_data=(val_img, y_val_one), verbose=1)
        
        #pulizia memoria e reset per il prossimo ciclo
        del X_train, y_train_one, X_val, y_val_one, train_img, val_img
        dataset = dataset.drop(samples.index)
        size = len(dataset)
        del samples, enc
        epoch = epoch + 1
        print("fine epoca " + str(epoch) + ";\trestano " + str(size) + " campioni nel dataset")
        
    ######## ULTIMO CICLO CHE FINISCE IL DATASET
    samples = dataset.sample(n = size)
    # splitting test and validation
    X_train, X_val, y_train, y_val = train_test_split(samples["image_id"], samples["dx"], test_size=0.30)
    
    train_img = np.asarray(load_images(X_train))
    val_img = np.asarray(load_images(X_val))
    
    #1hot encoder
    enc = OneHotEncoder(sparse=False)
    y_train_shaped = np.array(y_train).reshape(-1,1)
    y_val_shaped = np.array(y_val).reshape(-1,1)
    enc.fit(y_train_shaped)
    y_train_one = enc.transform(y_train_shaped)
    y_val_one = enc.transform(y_val_shaped)
    
    cl1.fit(train_img, y_train_one, epochs=epochs, shuffle=True, validation_data=(val_img, y_val_one), verbose=1)
    
    del X_train, y_train_one, X_val, y_val_one, train_img, val_img
    dataset = dataset.drop(samples.index)
    size = len(dataset)
    del samples, enc
    epoch = epoch + 1
    print("fine ultima epoca")

    cl1.save(classifier)
    return

def kaggle_classifier():
    # Set the CNN model 
    # my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
    input_shape = (150, 200, 3)
    build_shape = (None, 150, 200, 3)
    num_classes = 7
    
    model = Sequential()
    model.add(BatchNormalization())
    model.add(Conv2D(16, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(16,kernel_size=(3, 3), activation='relu',padding = 'Same',))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(32, (3, 3), activation='relu',padding = 'Same'))
    model.add(Conv2D(32, (3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.40))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.build(build_shape)
    model.summary()
    model.compile(optimizer='adadelta', loss='binary_crossentropy')
    return model

In [5]:
# - dataset splittato 80 20
dataset = pd.read_csv(csv_completo_2, encoding = "ISO-8859-1")

mass = int(len(dataset)*0.8)
train = dataset.sample(n=mass)
train.to_csv("v01_train08.csv", index=False)
test = dataset.drop(train.index)
test.to_csv("v01_test02.csv", index=False)

del test

In [4]:
train = pd.read_csv("v01_train08.csv", encoding = "ISO-8859-1")

In [11]:
# - creo cl_batch e cl_fit
cl_k = kaggle_classifier()
cl_k.save('v01_cl_kaggle.h5')


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_5 (Batch (None, 150, 200, 3)       12        
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 150, 200, 16)      448       
_________________________________________________________________
batch_normalization_6 (Batch (None, 150, 200, 16)      64        
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 150, 200, 16)      2320      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 75, 100, 16)       0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 75, 100, 16)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 75, 100, 32)      

In [5]:
# - carico in memoria le immagini
# - suddivido in train validation
# - 1hot encoding delle y
images = load_images(train["image_id"])

X_train, X_val, y_train, y_val = train_test_split(images, train["dx"], test_size=0.10)

#1hot encoder
enc = OneHotEncoder(sparse=False)
y_train_shaped = np.array(y_train).reshape(-1,1)
y_val_shaped = np.array(y_val).reshape(-1,1)
enc.fit(y_train_shaped)
y_train_one = enc.transform(y_train_shaped)
y_val_one = enc.transform(y_val_shaped)

In [6]:
# FIT kaggle
cl_name = "v01_cl_kaggle.h5"
cl_k = k.models.load_model(cl_name)
cl_k.fit(X_train, y_train_one, epochs=5, shuffle=True, validation_data=(X_val, y_val_one), verbose=1)
cl_k.save(cl_name)

del X_train, X_val, y_train, y_val, y_train_shaped, y_val_shaped, y_train_one, y_val_one

Train on 34527 samples, validate on 3837 samples
Epoch 1/5
34527/34527 [==============================] - 3704s 107ms/step - loss: 0.3142 - val_loss: 0.2568
Epoch 2/5
34527/34527 [==============================] - 3670s 106ms/step - loss: 0.2532 - val_loss: 0.2359
Epoch 3/5
34527/34527 [==============================] - 5256s 152ms/step - loss: 0.2307 - val_loss: 0.2121
Epoch 4/5
34527/34527 [==============================] - 1520s 44ms/step - loss: 0.2109 - val_loss: 0.2033
Epoch 5/5
34527/34527 [==============================] - 3192s 92ms/step - loss: 0.1967 - val_loss: 0.2163


In [8]:
############ TEST
# - carico in memoria il classificatore
# - istanzio l'encoder e lo fitto

#dataset = pd.read_csv(csv_completo_2, encoding = "ISO-8859-1")
testset = pd.read_csv("v01_test02.csv", encoding = "ISO-8859-1")

enc = OneHotEncoder(sparse=False)
y_train_shaped = np.array(testset["dx"]).reshape(-1,1)
enc.fit(y_train_shaped)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=False)

In [9]:
# - carico in memoria le immagini test

test_img = load_images(testset["image_id"])

In [10]:
# - predict del classificatore k

predictions1 = cl_k.predict(test_img)
pred_reverse1 = enc.inverse_transform(predictions1)
print(sklearn.metrics.classification_report(testset["dx"], pred_reverse1))
print('F1-score: '+ str(round(sklearn.metrics.f1_score(testset["dx"], pred_reverse1, average='micro'),2)))

              precision    recall  f1-score   support

           0       0.54      0.65      0.59      1391
           1       0.66      0.60      0.63      1104
           2       0.63      0.44      0.52      1567
           3       0.85      0.67      0.75      1345
           4       0.49      0.81      0.61      1507
           5       0.86      0.72      0.79      1329
           6       0.98      0.88      0.93      1348

    accuracy                           0.68      9591
   macro avg       0.72      0.68      0.69      9591
weighted avg       0.71      0.68      0.68      9591

F1-score: 0.68
